In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader 
from transformers import DistilBertTokenizer, DistilBertModel

In [15]:
!wget https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_train.csv

--2024-01-27 09:12:23--  https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_train.csv
Resolving ml-coding-test.s3.eu-west-1.amazonaws.com (ml-coding-test.s3.eu-west-1.amazonaws.com)... 52.218.90.248, 3.5.70.193, 52.218.116.234, ...
Connecting to ml-coding-test.s3.eu-west-1.amazonaws.com (ml-coding-test.s3.eu-west-1.amazonaws.com)|52.218.90.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93472103 (89M) [text/csv]
Saving to: 'webis_train.csv.1'

webis_train.csv.1   100%[===================>]  89.14M  16.4MB/s    in 7.1s    

2024-01-27 09:12:31 (12.6 MB/s) - 'webis_train.csv.1' saved [93472103/93472103]



In [3]:
df=pd.read_csv('/kaggle/working/webis_train.csv')

In [16]:
df

,Unnamed: 0,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription,truthJudgments,truthMean,truthClass,truthMedian,truthMode,cleaned_string
0,4795,[],Tensions are high in Jewish communities across...,833993791157723136,"['Jewish cemetery vandalized, JCCs threatened'...",['(CNN)Tensions remain high in Jewish communit...,Vandals damage 100 headstones at Jewish cemete...,Tue Feb 21 10:56:29 +0000 2017,"us, Vandals damage 100 headstones at Jewish ce...",Tensions remain high in Jewish communities acr...,"[0.0, 1.0, 0.0, 0.0, 0.0]",0.200000,no-clickbait,0.000000,0.000000,['(cnn)tensions remain high in jewish communit...
1,4979,[],"Turkey dismisses 4,400 public servants in late...",829279322729021440,['Turkish President Erdogan reviews a guard of...,['Dismissals come hours after first phone call...,"Turkey dismisses 4,400 public servants in late...",Wed Feb 08 10:42:52 +0000 2017,"Turkey,Recep Tayyip Erdoğan,UK news,Donald Tru...",Dismissals come hours after first phone call b...,"[0.0, 0.0, 0.33333333330000003, 0.0, 0.0]",0.066667,no-clickbait,0.000000,0.000000,['dismissals come hours after first phone call...
2,1448,['media/photo_818125834103857152.jpg'],"21 ""Law And Order"" Tumblr posts that are fucki...",818185517921632256,[],"['If SVU isn’t your favorite, are you even a r...","21 ""Law And Order"" Tumblr Posts That Are Fucki...",Sun Jan 08 20:00:03 +0000 2017,NaN,NaN,"[0.6666666666000001, 0.33333333330000003, 1.0,...",0.600000,clickbait,0.666667,1.000000,"['if svu isn’t your favorite, are you even a r..."
3,381,[],How one woman is using Tinder to take advantag...,847327280061919232,['istock-476739206.jpg'],['One user has received money from more than 2...,How women are using their Tinder matches to sc...,Thu Mar 30 05:59:00 +0000 2017,"Tinder, dating apps, Dating, scam, Online dati...",How to create the perfect Tinder bio is one of...,"[0.33333333330000003, 0.0, 1.0, 1.0, 0.6666666...",0.600000,clickbait,0.666667,1.000000,['one user has received money from more than 2...
4,7226,[],How the dinosaurs REALLY died out,820869672488890368,"[""Researchers say the 'big chill' had far more...",['The apocalyptic time that many believe wiped...,Is this how the dinosaurs REALLY died out? Res...,Mon Jan 16 05:45:55 +0000 2017,"A,dark,frozen,Earth,Researchers,reconstruct,ap...",Researchers say the 'big chill' had far more c...,"[0.33333333330000003, 1.0, 0.0, 1.0, 1.0]",0.666667,clickbait,1.000000,1.000000,['the apocalyptic time that many believe wiped...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9756,1296,['media/photo_822251094533292032.jpg'],This is what 100 years of women’s protest look...,822420412340387840,[],['“Life’s a bitch. You’ve got to go out and ki...,This Is What 100 Years Of Women’s Protest Look...,Fri Jan 20 12:28:00 +0000 2017,NaN,NaN,"[1.0, 0.33333333330000003, 0.33333333330000003...",0.466667,no-clickbait,0.333333,0.333333,['“life’s a bitch. you’ve got to go out and ki...
9757,575,['media/photo_842072978926010368.jpg'],One #WBC2017 fan’s attempt to catch a home run...,842074484026798080,[],['When\xa0Yurendell de Caster of the Netherlan...,Netherlands Home Run Hits World Baseball Class...,Wed Mar 15 18:06:16 +0000 2017,"MLB, Breaking News",When Yurendell de Caster of the Netherlands la...,"[0.33333333330000003, 0.6666666666000001, 0.33...",0.466667,no-clickbait,0.333333,0.333333,['when yurendell de caster of the netherlands ...
9758,840,[],Hunt master investigated by police after alleg...,834432577184292864,['Hunt master investigated after allegedly whi...,['A hunt master has been reported to police af...,Hunt master investigated by police after alleg...,Wed Feb 22 16:00:04 +0000 2017,"News,Surrey,Surrey Police,Hunting,Standard,UK ...",A hunt master has been reported to police afte...,"[0.33333333330000003, 0.0, 0.6666666666000001,...",0.400000,no-clickbait,0.333333,0.333333,['a hunt master has been reported to police af...
9759,3527,[],"""I

In [4]:
import pandas as pd

# Assuming df is your DataFrame with 'truthClass' column
df_clickbait = df[df['truthClass'] == 'clickbait']
df_no_clickbait = df[df['truthClass'] == 'no-clickbait'].head(5000)

# Concatenate the balanced DataFrames
balanced_df = pd.concat([df_clickbait, df_no_clickbait])

# Shuffle the DataFrame to mix the classes
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the value counts for each class
print(balanced_df['truthClass'].value_counts())

truthClass
no-clickbait    5000
clickbait       4761
Name: count, dtype: int64


In [5]:
df=balanced_df

In [6]:
import re

import pandas as pd
from concurrent.futures import ProcessPoolExecutor

In [7]:
def preprocess_text(text):
    cleaned_text = re.sub(r'\\n|\\xa0|\\', ' ', text)
    cleaned_text=cleaned_text.lower()
    return cleaned_text

# Load spaCy English language model


# Load spaCy stop words


# Batch processing using parallelism
batch_size = 100  # Adjust based on your data size
num_batches = len(df) // batch_size + 1

with ProcessPoolExecutor() as executor:
    result = list(executor.map(preprocess_text, df['targetParagraphs']))

# Add the cleaned string to the DataFrame
df['cleaned_string'] = result

# Print the cleaned DataFrame
print(df[['targetParagraphs', 'cleaned_string']])

                                       targetParagraphs  \
0     ['(CNN)Tensions remain high in Jewish communit...   
1     ['Dismissals come hours after first phone call...   
2     ['If SVU isn’t your favorite, are you even a r...   
3     ['One user has received money from more than 2...   
4     ['The apocalyptic time that many believe wiped...   
...                                                 ...   
9756  ['“Life’s a bitch. You’ve got to go out and ki...   
9757  ['When\xa0Yurendell de Caster of the Netherlan...   
9758  ['A hunt master has been reported to police af...   
9759  ['This year, rather than making best-of lists,...   
9760  ["We are closer than we have ever been to the ...   

                                         cleaned_string  
0     ['(cnn)tensions remain high in jewish communit...  
1     ['dismissals come hours after first phone call...  
2     ['if svu isn’t your favorite, are you even a r...  
3     ['one user has received money from more than 2...  
4

In [9]:
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd

In [22]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd
import torch

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')  # if GPU is available
# Load validation data

val_texts = df['cleaned_string'].tolist()
val_labels = df['truthClass'].map({'no-clickbait': 0, 'clickbait': 1}).tolist()  # convert sentiment to numeric
val_texts=val_texts[1000:6000]
val_labels=val_labels[1000:6000]

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Predict with the model


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification#, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.optim import AdamW


import time

# Record start time
start_time = time.time()

# Load data
  # assuming sentiment is encoded as 0 (negative) and 1 (positive)
reviews=val_texts
labels=val_labels
# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2)

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloaders
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
import time
from tqdm import tqdm
import torch

# Assuming you have a model, optimizer, and train_loader defined

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model.to(device)

# Number of epochs
num_epochs = 3

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Record start time
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    
    # Create a tqdm progress bar for the training loader
    tqdm_train_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', dynamic_ncols=True)
    
    total_loss = 0.0
    num_batches = len(train_loader)
    
    for batch in tqdm_train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Update tqdm progress bar
        tqdm_train_loader.set_postfix({'Loss': loss.item()})
    
    # Calculate and display average loss after each epoch
    average_loss = total_loss / num_batches
    print(f'Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss:.4f}')

# Save the model
model.save_pretrained('sentiment_model_DistilBERT')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3: 100%|██████████| 250/250 [01:47<00:00,  2.32it/s, Loss=0.641]


Epoch 1/3 - Average Loss: 0.6182


Epoch 2/3: 100%|██████████| 250/250 [01:50<00:00,  2.27it/s, Loss=0.46] 


Epoch 2/3 - Average Loss: 0.5258


Epoch 3/3: 100%|██████████| 250/250 [01:50<00:00,  2.27it/s, Loss=0.35] 


Epoch 3/3 - Average Loss: 0.4169
Time required to fine-tune:  328.65401220321655


In [10]:
from tqdm import tqdm 

In [16]:
for epoch in range(3):  # number of epochs
    model.train()
    running_loss = 0.0
    
    # Wrap the train_loader with tqdm for a progress bar
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{3}', leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # Display live-updating loss
        
    
    # Print average loss after each epoch
    avg_loss = running_loss / len(train_loader)
    print(f'\nEpoch {epoch + 1}/{3}, Avg Loss: {avg_loss:.4f}')

# Save the model
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)


Epoch 1/3, Avg Loss: 0.1516



Epoch 2/3, Avg Loss: 0.1163



Epoch 3/3, Avg Loss: 0.0950
Time required to fine-tune:  3734.223074913025


In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification#, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.optim import AdamW

import time
# Record start time
start_time = time.time()


# Load data
 # assuming sentiment is encoded as 0 (negative) and 1 (positive)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(val_texts, val_labels, test_size=0.2)

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloaders
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Evaluate the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

NameError: name 'DistilBertTokenizerFast' is not defined

In [18]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Evaluate the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

Accuracy: 0.7142857142857143
F1-score: 0.6934065934065934
Confusion matrix:
 [[764 235]
 [323 631]]


In [24]:
for epoch in range(20):
    model.train()
    
    # Create a tqdm progress bar for the training loader
    tqdm_train_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', dynamic_ncols=True)
    
    total_loss = 0.0
    num_batches = len(train_loader)
    
    for batch in tqdm_train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Update tqdm progress bar
        tqdm_train_loader.set_postfix({'Loss': loss.item()})
    
    # Calculate and display average loss after each epoch
    average_loss = total_loss / num_batches
    print(f'Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss:.4f}')

# Save the model
model.save_pretrained('sentiment_model_DistilBERT')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)



Epoch 1/3: 100%|██████████| 250/250 [01:50<00:00,  2.27it/s, Loss=0.172] 


Epoch 1/3 - Average Loss: 0.2622


Epoch 2/3: 100%|██████████| 250/250 [01:50<00:00,  2.27it/s, Loss=0.414]  


Epoch 2/3 - Average Loss: 0.1232


Epoch 3/3: 100%|██████████| 250/250 [01:50<00:00,  2.27it/s, Loss=0.408]  


...

Epoch 7/3 - Average Loss: 0.0360


Epoch 8/3: 100%|██████████| 250/250 [01:50<00:00,  2.27it/s, Loss=0.00807] 


Epoch 8/3 - Average Loss: 0.0369


Epoch 9/3:  95%|█████████▌| 238/250 [01:45<00:05,  2.26it/s, Loss=0.0232]  


KeyboardInterrupt: 

In [25]:
import pickle

In [27]:
Pkl_Filename = "Pickle_RL_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)